In [73]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [74]:
#!pip install statsmodels

In [75]:
#importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
# import lazypredict

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

import statsmodels.tsa.stattools as st
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA

In [76]:
df = pd.read_csv('GCB2022v27_MtCO2_flat.csv')
df

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,Afghanistan,AFG,1750,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1751,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1752,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1753,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1754,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
63099,Global,WLD,2017,36096.739276,14506.973805,12242.627935,7144.928128,1507.923185,391.992176,302.294047,4.749682
63100,Global,WLD,2018,36826.506600,14746.830688,12266.016285,7529.846784,1569.218392,412.115746,302.478706,4.792753
63101,Global,WLD,2019,37082.558969,14725.978025,12345.653374,7647.528220,1617.506786,439.253991,306.638573,4.775633
63102,Global,WLD,2020,35264.085734,14174.564010,11191.808551,7556.290283,1637.537532,407.583673,296.301685,4.497423


In [77]:
#dropping years less than 1950
df.drop(df[df['Year'] < 1950].index, inplace = True)

In [78]:
df = df.reset_index()

In [79]:
df.columns

Index(['index', 'Country', 'ISO 3166-1 alpha-3', 'Year', 'Total', 'Coal',
       'Oil', 'Gas', 'Cement', 'Flaring', 'Other', 'Per Capita'],
      dtype='object')

In [80]:
## Filling nans with zero for the inpuut features
df.Coal = df['Coal'].fillna(0, inplace = False)
df.Oil = df['Oil'].fillna(0, inplace = False)
df.Gas = df['Gas'].fillna(0, inplace = False)
df.Cement = df['Cement'].fillna(0, inplace = False)
df.Flaring = df['Flaring'].fillna(0, inplace = False)
df.Other = df['Other'].fillna(0, inplace = False)
df['Per Capita'] = df['Per Capita'].fillna(0, inplace = False)

In [81]:
# Checking the count of null values
df.isna().sum()

index                   0
Country                 0
ISO 3166-1 alpha-3    432
Year                    0
Total                   0
Coal                    0
Oil                     0
Gas                     0
Cement                  0
Flaring                 0
Other                   0
Per Capita              0
dtype: int64

In [83]:
df.reset_index()
df

,index,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,200,Afghanistan,AFG,1950,0.084272,0.021068,0.063204,0.000000,0.000000,0.000000,0.000000,0.011266
1,201,Afghanistan,AFG,1951,0.091600,0.025648,0.065952,0.000000,0.000000,0.000000,0.000000,0.012098
2,202,Afghanistan,AFG,1952,0.091600,0.031708,0.059892,0.000000,0.000000,0.000000,0.000000,0.011946
3,203,Afghanistan,AFG,1953,0.106256,0.037949,0.068307,0.000000,0.000000,0.000000,0.000000,0.013685
4,204,Afghanistan,AFG,1954,0.106256,0.042502,0.063754,0.000000,0.000000,0.000000,0.000000,0.013511
...,...,...,...,...,...,...,...,...,...,...,...,...
16699,63099,Global,WLD,2017,36096.739276,14506.973805,12242.627935,7144.928128,1507.923185,391.992176,302.294047,4.749682
16700,63100,Global,WLD,2018,36826.506600,14746.830688,12266.016285,7529.846784,1569.218392,412.115746,302.478706,4.792753
16701,63101,Global,WLD,2019,37082.558969,14725.978025,12345.653374,7647.528220,1617.506786,439.253991,306.638573,4.775633
16702,63102,Global,WLD,2020,35264.085734,14174.564010,11191.808551,7556.290283,1637.537532,407.583673,296.301685,4.497423
